<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-phoenix-assets/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>
<h1 align="center">Root-Cause Analysis for a Drifting Sentiment Classification Model</h1>

Imagine you're in charge of maintaining a model that takes as input online reviews of your U.S.-based product and classifies the sentiment of each review as positive, negative, or neutral. Your model initially performs well in production, but its performance gradually degrades over time.

Phoenix helps you surface the reason for this regression by analyzing the embeddings representing the text of each review. Your model was trained on English reviews, but as you'll discover, it's encountering Spanish reviews in production that it can't correctly classify.

In this tutorial, you will:
* Download curated datasets of embeddings and predictions
* Define a schema to describe the format of your data
* Launch Phoenix to visually explore your embeddings
* Investigate problematic clusters to identify the root cause of your model performance issue

⚠️ This notebook runs slowly without a GPU. If you don't have access to a GPU, you can still use Phoenix by skipping the cells preceded by the 💬 emoji.

Let's get started!

## Install Dependencies and Import Libraries

Install Phoenix.

In [ ]:
!pip install "arize[AutoEmbeddings]" "arize-phoenix>=3.20.0"

Import dependencies.

In [ ]:
import pandas as pd
import phoenix as px
from arize.pandas.embeddings import EmbeddingGenerator, UseCases

## Download the Data

Download training and production data from a model that classifies the sentiment of product reviews as positive, negative, or neutral.

In [ ]:
train_df = pd.read_parquet(
    "http://storage.googleapis.com/arize-phoenix-assets/datasets/unstructured/nlp/sentiment-classification-language-drift/sentiment_classification_language_drift_training.parquet",
).reset_index(drop=True)
prod_df = pd.read_parquet(
    "http://storage.googleapis.com/arize-phoenix-assets/datasets/unstructured/nlp/sentiment-classification-language-drift/sentiment_classification_language_drift_production.parquet",
).reset_index(drop=True)

View a few training data points.

In [ ]:
train_df.head()

The columns of the dataframe are:
- **prediction_ts:** the Unix timestamps of your predictions
- **review_age**, **reviewer_gender**, **product_category**, **language:** the features of your model
- **text:** the text of each product review
- **text_vector:** the embedding vectors representing each review
- **pred_label:** the label your model predicted
- **label:** the ground-truth label for each review

## Compute Embeddings

💬 Compute embeddings using a [DistilBERT model fine-tuned on a dataset of product reviews](https://huggingface.co/arize-ai/distilbert_reviews_with_language_drift).

In [ ]:
model_name = "arize-ai/distilbert_reviews_with_language_drift"
generator = EmbeddingGenerator.from_use_case(
    use_case=UseCases.NLP.SEQUENCE_CLASSIFICATION,
    model_name=model_name,
)
train_df["text_vector"] = generator.generate_embeddings(text_col=train_df["text"])
prod_df["text_vector"] = generator.generate_embeddings(text_col=prod_df["text"])

## Launch Phoenix

Define a schema to tell Phoenix what the columns of your dataframe represent (predictions, actuals, embeddings, etc.). See the [docs](https://docs.arize.com/phoenix/) for guides on how to define your own schema and API reference on `phoenix.Schema` and `phoenix.EmbeddingColumnNames`.

In [ ]:
schema = px.Schema(
    prediction_id_column_name="prediction_id",
    timestamp_column_name="prediction_ts",
    prediction_label_column_name="pred_label",
    actual_label_column_name="label",
    embedding_feature_column_names={
        "text_embedding": px.EmbeddingColumnNames(
            vector_column_name="text_vector", raw_data_column_name="text"
        ),
    },
)

Create Phoenix datasets that wrap your dataframes with schemas that describe them.

In [ ]:
prod_ds = px.Inferences(dataframe=prod_df, schema=schema, name="production")
train_ds = px.Inferences(dataframe=train_df, schema=schema, name="training")

Launch Phoenix. Follow the instructions in the cell output to open the Phoenix UI in your notebook or in a separate browser tab.

In [ ]:
session = px.launch_app(primary=prod_ds, reference=train_ds)

## Find the Root-Cause of Your Model Performance Issue

Click on "text_embedding" in the "Embeddings" section.

![click on text embedding](http://storage.googleapis.com/arize-phoenix-assets/assets/docs/notebooks/sentiment-classification-tutorial/click_on_text_embedding.png)


In the Euclidean distance graph at the top of the page, click a point on the graph where the Euclidean distance is high.

![select period of high drift](http://storage.googleapis.com/arize-phoenix-assets/assets/docs/notebooks/sentiment-classification-tutorial/select_period_of_high_drift.png)

Click on the top cluster in the panel on the left.

![select top cluster](http://storage.googleapis.com/arize-phoenix-assets/assets/docs/notebooks/sentiment-classification-tutorial/select_top_cluster.png)

Use the panel at the bottom to examine the data points in this cluster.

![inspect points in cluster](http://storage.googleapis.com/arize-phoenix-assets/assets/docs/notebooks/sentiment-classification-tutorial/inspect_points_in_cluster.png)

What do you notice about the text in this cluster? Select other clusters and compare the text. Do you notice a difference?

It turns out that your model is seeing Spanish product reviews in production, but the training data is all in English. Congrats! You've identified the root-cause of the issue. As an actionable next step, you should enhance your model to support other languages, e.g., by fine-tuning on Spanish product reviews.